## import

In [6]:
!apt update

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease   
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (119 kB/s)0m 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
All packages are up to date.


In [7]:
!apt upgrade -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Calculating upgrade... Done
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [1]:
!apt install gcc python3-dev python3-pip libxml2-dev libxslt1-dev zlib1g-dev g++ -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
gcc is already the newest version (4:7.4.0-1ubuntu2.3).
The following additional packages will be installed:
  build-essential dh-python dirmngr dpkg-dev fakeroot g++-7
  gir1.2-harfbuzz-0.0 gnupg gnupg-l10n gnupg-utils gpg gpg-agent
  gpg-wks-client gpg-wks-server gpgconf gpgsm icu-devtools
  libalgorithm-diff-perl libalgorithm-diff-xs-perl libalgorithm-merge-perl
  libassuan0 libdpkg-perl libelf1 libexpat1-dev libfakeroot
  libfile-fcntllock-perl libfreetype6 libglib2.0-bin libglib2.0-dev
  libglib2.0-dev-bin libgraphite2-3 libgraphite2-dev libharfbuzz-dev
  libharfbuzz-gobject0 libharfbuzz-icu0 libharfbuzz0b libicu-dev
  libicu-le-hb-dev libicu-le-hb0 libiculx60 libksba8 liblocale-gettext-perl
  libnpth0 libpcre16-3 libpcre3-dev libpcre32-3 libpcrecpp0v5 libpython3-dev
  libpython3.6-d

In [2]:
!pip install git+https://github.com/lucasb-eyer/pydensecrf.git

  Cloning https://github.com/lucasb-eyer/pydensecrf.git to /tmp/pip-req-build-dedkxbaq
  Running command git clone -q https://github.com/lucasb-eyer/pydensecrf.git /tmp/pip-req-build-dedkxbaq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pydensecrf: filename=pydensecrf-1.0rc2-cp37-cp37m-linux_x86_64.whl size=2898466 sha256=d25717ea6357fbe1ab06e13cd4e00c7de33950a6222dcbd15515ea9e56ba6188
  Stored in directory: /tmp/pip-ephem-wheel-cache-fjld41th/wheels/c1/7e/80/99adc0b2f215180486e24dd9c700028343ba5f566514a0ef05
Successfully built pydensecrf


In [3]:
import os
import random
import time
import json
import warnings 
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from utils import *
import cv2
from sklearn.model_selection import StratifiedKFold
import zipfile

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

# Pretrained Model
import segmentation_models_pytorch as smp

# CRF
import pydensecrf.densecrf as dcrf
import pydensecrf.utils as utils

# torchvision Models
import torchvision
from torchvision import models
from torchvision.models.segmentation.deeplabv3 import DeepLabHead

# 전처리를 위한 라이브러리
from torch.utils.data.sampler import WeightedRandomSampler
from pycocotools.coco import COCO
import torchvision
import torchvision.transforms as transforms

import albumentations as A
from albumentations.pytorch import ToTensorV2

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

plt.rcParams['axes.grid'] = False

print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))

print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())

device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장
%matplotlib inline

pytorch version: 1.5.1+cu101
GPU 사용 가능 여부: True
Tesla V100-PCIE-32GB
1


## 하이퍼파라미터 세팅 및 seed 고정

In [4]:
batch_size = 20  # Mini-batch size
num_epochs = 40
learning_rate = 1e-4

In [5]:
# seed 고정
random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
# torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [6]:
class CustomDataLoader(Dataset):
    """COCO format"""
    def __init__(self, data_dir, mode = 'test', transform = None):
        super().__init__()
        self.mode = mode
        self.transform = transform
        self.coco = COCO(data_dir)
        
    def __getitem__(self, index: int):
        # dataset이 index되어 list처럼 동작
        image_id = self.coco.getImgIds(imgIds=index)
        image_infos = self.coco.loadImgs(image_id)[0]
        
        # cv2 를 활용하여 image 불러오기
        images = cv2.imread(os.path.join('../input/data', image_infos['file_name']))
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB)
        
        if self.mode == 'test':
            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images)
                images = transformed["image"]
            
            return images, image_infos
    
    
    def __len__(self) -> int:
        # 전체 dataset의 size를 return
        return len(self.coco.getImgIds())

## Dataset 정의 및 DataLoader 할당

In [7]:
# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))

test_transform = A.Compose([
                            A.Normalize(
                                mean=(0.485, 0.456, 0.406),
                                std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0
                            ),
                            ToTensorV2(),
                           ])

In [8]:
test_dataset = CustomDataLoader('../input/data/test.json', mode="test", transform=test_transform)

test_loader = DataLoader(test_dataset, 
                          batch_size=1, 
                          num_workers=4,
                         shuffle=False,
                         collate_fn=collate_fn)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


## Inference Load Model

In [19]:
MODEL_PATHS = ['../single_best_weight/0_checkpoint.pt',
                '../single_best_weight/1_checkpoint.pt',
                '../single_best_weight/2_checkpoint.pt',
                '../single_best_weight/3_checkpoint.pt',
                '../single_best_weight/4_checkpoint.pt']

OUT_MASKS = f'./saved/test_masks.zip'

SOFTMAX = nn.Softmax(dim=1)

In [26]:
MAX_ITER = 5
POS_W = 3
POS_XY_STD = 1
Bi_W = 4
Bi_XY_STD = 67
Bi_RGB_STD = 3

def dense_crf(img, prob):
    """https://github.com/zllrunning/deeplab-pytorch-crf"""
    c, h, w = prob.shape
    U = utils.unary_from_softmax(prob)
    U = np.ascontiguousarray(U)
    img = np.ascontiguousarray(img)
    d = dcrf.DenseCRF2D(w, h, c)
    d.setUnaryEnergy(U)
    d.addPairwiseGaussian(sxy=POS_XY_STD, compat=POS_W)
    d.addPairwiseBilateral(sxy=Bi_XY_STD, srgb=Bi_RGB_STD, rgbim=img, compat=Bi_W)
    Q = d.inference(MAX_ITER)
    return np.array(Q).reshape((c, h, w))

In [27]:
def inference(models, imgs, device):
    outs = None
    flip_outs = None
    flips = [[-1],[-2],[-2,-1]]
    for model in models:
        model.eval()
        if outs == None:
            outs = model(imgs.to(device).float()).detach()
        else:
            outs += model(imgs.to(device).float()).detach()
             
    for flip in flips:
        flip_img = torch.flip(imgs, flip)
        tmp_outs = None
        for model in models:
            flip_out = model(flip_img.to(device).float()).detach()
            flip_out = torch.flip(flip_out, flip)
            if tmp_outs == None:
                tmp_outs = flip_out
            else:
                tmp_outs += flip_out
        if flip_outs == None:
            flip_outs = tmp_outs
        else:
            flip_outs += tmp_outs
    
    outs += flip_outs
    
    return outs/(len(models)+len(flips)+len(models))

## Inference

In [28]:
def test(models, data_loader, device):
    size = 256
    transform = A.Compose([A.Resize(256, 256)])
    print('Start prediction.')
    file_name_list = []
    preds_array = np.empty((0, size*size), dtype=np.long)
    pbar = tqdm(enumerate(data_loader), total=len(data_loader), position=0, leave=True)
    with torch.no_grad():
        with zipfile.ZipFile(OUT_MASKS, 'w') as mask_out:
            for step, (imgs, image_infos) in pbar:
                # print(imgs)
                # print(imgs)
                imgs = torch.stack(imgs)

                outs = SOFTMAX(inference(models, imgs, device))

                image = imgs.squeeze().data.cpu().numpy().astype(np.uint8).transpose(1,2,0)
                prob = outs.squeeze().data.cpu().numpy()

                prob = dense_crf(image,prob)
                
                outs = torch.Tensor(prob).unsqueeze(0)

                oms = torch.argmax(outs, dim=1).detach().cpu().numpy()
                
                file_name = image_infos[0]['file_name'].split("/")
                file_name[0] += "_masks"
                file_name[1] = file_name[1][:-4]
                file_name = "/".join(file_name)

                m = cv2.imencode(".png", oms.squeeze())[1]
                mask_out.writestr(f"{file_name}.png", m)

                # resize (256 x 256)
                temp_mask = []
                for img, mask in zip(np.stack(imgs), oms):
                    # print(mask.shape)
                    transformed = transform(image=img, mask=mask)
                    mask = transformed['mask']
                    temp_mask.append(mask)

                oms = np.array(temp_mask)
                oms = oms.reshape([oms.shape[0], size*size]).astype(int)

                preds_array = np.vstack((preds_array, oms))

                file_name_list.append([i['file_name'] for i in image_infos])
    print("End prediction.")
    file_names = [y for x in file_name_list for y in x]
    
    return file_names, preds_array

In [29]:
models=[]
for path in MODEL_PATHS:
    model = smp.DeepLabV3Plus('resnext50_32x4d', encoder_weights=None, classes=12)
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint)
    model.eval()
    model.to(device)
    models.append(model)
    
    del checkpoint

In [30]:
file_names, preds = test(models, test_loader, device)

Start prediction.


  0%|          | 0/837 [00:00<?, ?it/s]

End prediction.


## submission.csv 생성

In [31]:
# sample_submisson.csv 열기
submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)

# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv("./submission/psheudo_3_CRF.csv", index=False)

## Reference

